In [1]:
# 필수 라이브러리 호출
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score as acc_sc

In [7]:
# 데이터 정의

st_default_org = pd.read_csv('./ysp_files/st_default.csv')
st_default_df = st_default_org.iloc[:,1:] # iloc는 위치기반 인덱싱(positional indexing)

In [11]:
# 전체 데이터 
st_default_df

# X_data와 y_target을 정의한다.
X_data = st_default_df.drop('default', axis=1) # st_default_df.iloc[:,1:]
y_target = st_default_df.default

In [13]:
# EDA를 통한 데이터 확인
X_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   student  10000 non-null  object 
 1   balance  10000 non-null  float64
 2   income   10000 non-null  float64
dtypes: float64(2), object(1)
memory usage: 234.5+ KB


In [15]:
X_data.describe()  # 통계치

,balance,income
count,10000.000000,10000.000000
mean,835.374886,33516.981876
std,483.714985,13336.639563
min,0.000000,771.967729
25%,481.731105,21340.462905
50%,823.636973,34552.644800
75%,1166.308387,43807.729275
max,2654.322576,73554.233500


In [17]:
# 크기확인
print('X_data의 형태:', X_data.shape)
print('y_target의 형태:', y_target.shape)

X_data의 형태: (10000, 3)
y_target의 형태: (10000,)


In [19]:
# 문자들을 숫자로 변환한다. - 1) 그냥 문자(학생) 컬럼 없애면? O.K. vs 
#                           2) 문자 -> one-hot // Label Encoding 

X_data_df = pd.get_dummies(X_data) # one-hot :: 문자 -> 레이블 -> 원핫 : ndarray

In [21]:
y_target[:5] # Series

0    No
1    No
2    No
3    No
4    No
Name: default, dtype: object

In [28]:
# y 값에는 Label Encoding(레이블 인코딩)
from sklearn.preprocessing import LabelEncoder

le_encode = LabelEncoder()
y_target_scaled = le_encode.fit_transform(y_target)

# y_target값을 확인 - imbalanced_data(쏠림 데이터)
pd.Series(y_target_scaled).value_counts()

333/10000 # 3.33% 부도낸 사람

0.0333

In [31]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_data_df, y_target_scaled, stratify=y_target_scaled)

In [32]:
X_train

,balance,income,student_No,student_Yes
1846,873.641237,21810.61199,0,1
766,1027.895035,21551.61109,1,0
5600,592.432321,32283.26412,1,0
98,243.841328,47193.88813,1,0
6207,617.640764,37604.93356,1,0
...,...,...,...,...
4744,0.000000,45486.45868,1,0
1785,396.513587,41969.74677,1,0
9706,1070.752120,11383.44265,0,1
2390,1264.586941,14649.15502,0,1


In [36]:
# 의사결정나무 적용
dt_clf = DecisionTreeClassifier()
dt_clf.fit(X_train,y_train) # 학습을 수행

# 예측
y_pred = dt_clf.predict(X_test)

In [37]:
# 평가지표
print(f'정확도는:{acc_sc(y_pred, y_test)}')

정확도는:0.954


In [42]:
# CV - 교차검증을 수행 :: cross_val_score
from sklearn.model_selection import cross_val_score

# 전체 데이터 얼마나 CV의 정확도??
scores = cross_val_score(dt_clf, X_train, y_train, scoring='accuracy', cv=5)

print(f'교차 검증별 정확도:{np.round(scores,4)}')
print(f'평균 검증별 정확도:{np.round(np.mean(scores),4)}')

교차 검증별 정확도:[0.9507 0.9613 0.9567 0.9567 0.954 ]
평균 검증별 정확도:0.9559


In [44]:
y_target.value_counts()

No     9667
Yes     333
Name: default, dtype: int64

In [ ]:
# 1. 왜 데이터의 해당하는 정확도 값이 높게 나왔을까요?
# 2. 데이터는 어떠한 특징을 지니고 있나요?
# 3. 데이터에는 어떠한 문제점이 있나요?
# 4. 그렇다면 이러한 문제점을 해결하기 위한 방안은 무엇이 있을까요?

# [1] oversampling vs undersampling
# [2] Data augementation - 이미지 분석 
# [3] get more data
# [4] change our metrics - precision / recall / f1-score / roc_auc

In [45]:
# end of file